In [5]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision
import torchvision.transforms as T

import librosa
import librosa.display as display
import IPython.display as ipd
from IPython.display import Audio

import torchaudio
import torchaudio.transforms as AT
import torchaudio.functional as AF

import random
from torchsummary import summary

from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#최종 형태 나오고 -> 정규화

cuda


In [2]:
dist_util.setup_dist()

setup_dist start
os.environ[CUDA_VISIBLE_DEVICES]: 0
setup_dist


In [3]:
audio_mnist_path = sorted(glob.glob("/data/yjpak/Dataset/audio_mnist/*/*.wav"))
audio_mnist_label = [] 
for i in range(len(audio_mnist_path)):
    audio_mnist_label.append(int(audio_mnist_path[i].split('/')[6].split('_')[0]))
    # print(int(audio_mnist_path[i].split('/')[6].split('_')[0]))
train_path = audio_mnist_path
train_label = audio_mnist_label

In [4]:
np.unique(audio_mnist_label, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]))

In [17]:
class CNN_2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),   
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),   
            
        )
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(256,10)
        
    def forward(self, x):
        x = self.features(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0),-1)
        x = self.fc1(x)

        return x
    
model_2D = CNN_2D().to(device)

In [18]:
from torch.optim import Adam
criterion = nn.CrossEntropyLoss().to(device)
opti_2D = Adam(model_2D.parameters(), lr=1e-4)

In [19]:
def train(model, dataloader, criterion, data_len, opti):
    correct = 0
    total_loss = 0
    model.train()
    
    for data, target in dataloader:
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)

        opti.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        opti.step()

        total_loss += loss.item()
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()

    acc = correct / data_len
    total_loss = total_loss / data_len
    
#     scheduler.step()
    
    return total_loss, acc

In [ ]:
random_seed = 42
torch.cuda.manual_seed(random_seed)
max_acc = 0

from tqdm.auto import tqdm
epoch = 5

mel_train_loss_list = []
mel_test_loss_list = []
mel_train_acc_list = []
mel_test_acc_list = []


for i in tqdm(range(epoch)):
    '''
    MelSpectrogram
    '''
    
    mel_train_loss, mel_train_acc = train(model_2D, mel_train_dataloader, criterion, len(mel_train_dataloader.dataset), opti_2D)
    mel_test_loss, mel_test_acc = evaluate(model_2D, mel_test_dataloader, criterion, len(mel_test_dataloader.dataset))
    
    mel_train_loss_list.append(mel_train_loss)
    mel_test_loss_list.append(mel_test_loss)
    mel_train_acc_list.append(mel_train_acc)
    mel_test_acc_list.append(mel_test_acc)
    
    print(f"Case: Mel\nepoch = {i+1} \ntrain_loss = {mel_train_loss:.4f}, train_acc = {mel_train_acc:.4f}, \ntest_loss = {mel_test_loss:.4f}, test_acc = {mel_test_acc:.4f}")
    